In [ ]:
# Instale os requisitos se ainda não tiver
#!pip install ultralytics opencv-python-headless --quiet

In [ ]:
# Load the YOLOv5n model (auto-downloads on first run)
#model = YOLO('yolov5n.pt')

In [1]:
# GUI: Real-Time Person Detection with Start/Stop Button (Tkinter + YOLOv5n)

import tkinter as tk
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO
import threading

model = YOLO('yolov5n.pt')

# Initialize GUI window
root = tk.Tk()
root.title("Person Detection Alert System")

video_label = tk.Label(root)
video_label.pack()

alert_label = tk.Label(root, text="Waiting...", font=("Arial", 16))
alert_label.pack(pady=10)

running = False
cap = None

# Function to detect people and update the GUI
def detect_loop():
    global running, cap
    while running:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (640, 480))
        results = model(frame, verbose=False)

        alert = False
        for box in results[0].boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            if cls == 0 and conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'Person {conf:.2f}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                alert = True

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(image=img)
        video_label.configure(image=img)
        video_label.image = img

        alert_label.config(
            text="\u26a0\ufe0f Person Detected!" if alert else "No person detected",
            fg="red" if alert else "green"
        )

# Start detection thread

def start_detection():
    global running, cap
    if not running:
        cap = cv2.VideoCapture(1)
        running = True
        threading.Thread(target=detect_loop, daemon=True).start()

# Stop detection and release resources

def stop_detection():
    global running, cap
    running = False
    if cap:
        cap.release()
    alert_label.config(text="Detection stopped", fg="black")
    video_label.config(image='')

# Buttons
start_button = tk.Button(root, text="Start Detection", command=start_detection, bg="green", fg="white")
start_button.pack(pady=5)

stop_button = tk.Button(root, text="Stop Detection", command=stop_detection, bg="red", fg="white")
stop_button.pack(pady=5)

# Run GUI loop
root.mainloop()

PRO TIP  Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

